# An Example of the Horizontal Logistic Regression Task

This is an example of running horizontal logistic regression task on multiple Delta Nodes.

The data is [spector dataset](https://www.statsmodels.org/stable/datasets/generated/spector.html), experimental data on the effectiveness of the personalized system of instruction (PSI) program.
The format of data is a csv file with 4 columns: Grade, TCUE, PSI and GPA. The task is to estimate student's GPA.

This example could be executed in Deltaboard directly. <span style="color:#FF8F8F;font-weight:bold">Before hitting the run button, the Delta Node API address should be modified according to the user's config, the instructions are explained in section 4 below.</span>

## 1. Import the Required Packages

We need to import Delta Task framework components from Python package ```delta-task``` including ```DeltaNode``` for Delta Node API connection, and the class  ```LogitTask``` for defination of the horizontal logistic regression task:

In [ ]:
import pandas
import delta.dataset
from delta import DeltaNode
from delta.statsmodel import LogitTask

## 2. Define the Horizontal Logistic Regression Task

The next step is to define our horizontal logistic regression task to analyze the data on multiple nodes.

There're several parts in the PPC Task that need to be programmed by the developer:

* ***Task Config***: We can make some basis task config in the ```super().__init__()``` method. The configurations involves task name (```name```), minimum client count(```min_clients```), maximum client count(```max_clients```),waiting timeout for calculation (```wait_timeout```)，and connection timeout for each step in the procedure(```connection_timeout```).

  And we can start the zero-knownledge proof step to verify the convergence of result and the consistence of data after the task is finished. To start the zero-knownledge proof step, we need to set `enable_verify` to True in `super().__init__()` method. And also, we could control the timeout of zero-knownledge proof step by parameter `verify_timeout`. Now the zero-knownledge proof step consumes pretty long time and the default value of `verify_timeout` is 300 second. If timeout error occurs in the the zero-knownledge proof step, we should set `verify_timeout` to a bigger value.

  ***We decide to disable the zero-knownledge proof step on online demo system due to the resouce restrictions. You should set `enable_verify` to False on online demo system.***

* ***Dataset***: In the ```dataset``` method, you can specify the dataset for task. The return value is a dict of which key should be the name of dataset and value should be an instance of ```delta.dataset.DataFrame```; the key of dict should be corresponding to the parameters of the execute method. For detailed explanation of the dataset format, please refer to [this document](https://docs.deltampc.com/network-deployment/prepare-data).
* ***Preprocess***: In the ```preprocess```, you need to preprocess the dataset, and finally return the x and y for the task. The input parameters should be the same with the keys of returned dict of ```dataset``` method. The returned x and y can be ```pandas.DataFrame``` or ```numpy.ndarray```, and y should be a 1-D array of data labels.
* ***Options***: This method is optional. In the ```option``` method, you can specify some options for the logistic regression. The general options are ```method``` (fit method for logistic regression, only `newton` is available now) and `maxiter` (max iterations for fit). The newton method has some specific options, including `ord` (the norm ord for the gradient), `tol` (the stopping tolerance) and `ridge_factor` (the ridge regression factor for the hessian matrices). All these options have default values. You don't need to implment this method unless you have special needs.


In [ ]:
class SpectorLogitTask(LogitTask):
    def __init__(
        self,
    ) -> None:
        super().__init__(
            name="spector_logit",  # The task name which is used for displaying purpose.
            min_clients=2,  # Minimum nodes required in each round, must be greater than 2.
            max_clients=3,  # Maximum nodes allowed in each round, must be greater equal than min_clients.
            wait_timeout=5,  # Timeout for calculation.
            connection_timeout=5,  # Wait timeout for each step.
            verify_timeout=500,  # Timeout for the final zero knownledge verification step
            enable_verify=False  # whether to enable final zero knownledge verification step
        )

    def dataset(self):
        return {
            "data": delta.dataset.DataFrame("spector.csv"),
        }

    def preprocess(self, data: pandas.DataFrame):
        names = data.columns

        y_name = names[3]
        y = data[y_name].copy()  # type: ignore
        x = data.drop([y_name], axis=1)
        return x, y


## 3. Set the API Address of the Delta Node

After defining the task details, we're ready to run the task on the Delta Nodes.

Delta Task framework could send the task to Delta Node directly, as long as the Delta Node API address is specified.

Here we use the Delta Node API provided by Deltaboard. Deltaboard provides a separate API address for each of its users, the tasks submitted via the API could be listed inside Deltaboard. The developer could also use API from Delta Node directly.

Click "Profiles" on the sidebar of Deltaboard, copy the API Address in Deltaboard API section, and paste it here:

In [ ]:
DELTA_NODE_API = "http://127.0.0.1:6704"

## 4. Run the PPC Task

Finally we can start the task:

In [ ]:
task = SpectorLogitTask().build()
delta_node = DeltaNode(DELTA_NODE_API)
delta_node.create_task(task)

## 5. Check the Running Status

After clicking the run button, some logs will be print out showing the task is submitted to the Delta Node successfully.

To see the task execution details, go to "My Tasks" on the sidebar of Deltaboard, the task should be listed.
Click the item to view the execution logs.